In [4]:
import yaml

with open("../configs/ensemble.yaml","r") as file:
    configs = yaml.safe_load(file)

configs

{'ClustalO': {'aligner': 'ClustalO',
  'type': 'default',
  'params': {'--iterations': 6,
   '--max-guidetree-iterations': [1, 2, 3, 4],
   '--max-hmm-iterations': [1, 2]}},
 'MAFFT_FFT_NS_2': {'aligner': 'MAFFT',
  'type': 'default',
  'params': {'--op': [1.1475, 1.4025, 1.6575, 1.9125],
   '--ep': [0.0922, 0.1538]}},
 'MAFFT_L_INS_I': {'aligner': 'MAFFT',
  'type': 'default',
  'params': {'--localpair': None,
   '--maxiterate': 1000,
   '--op': [1.1475, 1.4025, 1.6575, 1.9125],
   '--ep': [0.0922, 0.1538]}},
 'MAFFT_G_INS_I': {'aligner': 'MAFFT',
  'type': 'default',
  'params': {'--globalpair': None,
   '--maxiterate': 1000,
   '--op': [1.1475, 1.4025, 1.6575, 1.9125],
   '--ep': [0.0922, 0.1538]}},
 'Muscle3': {'aligner': 'Muscle3',
  'type': 'default',
  'params': {'-weight1': ['clustalw', 'none'],
   '-weight2': ['clustalw', 'none'],
   '-objscore': ['spf', 'xp']}}}

In [5]:
import pathlib
from ensemblify.utils import get_option_variants

print(pathlib.Path.cwd())
get_option_variants(configs["MAFFT_L_INS_I"]["params"])

/hits/fast/cme/bodynems/ensemblify/notebooks


['--localpair§--maxiterate§1000§--op§1.1475§--ep§0.0922',
 '--localpair§--maxiterate§1000§--op§1.1475§--ep§0.1538',
 '--localpair§--maxiterate§1000§--op§1.4025§--ep§0.0922',
 '--localpair§--maxiterate§1000§--op§1.4025§--ep§0.1538',
 '--localpair§--maxiterate§1000§--op§1.6575§--ep§0.0922',
 '--localpair§--maxiterate§1000§--op§1.6575§--ep§0.1538',
 '--localpair§--maxiterate§1000§--op§1.9125§--ep§0.0922',
 '--localpair§--maxiterate§1000§--op§1.9125§--ep§0.1538']

In [34]:
%reload_ext autoreload
%autoreload 2
from ensemblify.manager import DefaultManager, Manager, infer_manager
import yaml
import os
DATA_DIR = pathlib.Path("/hits/fast/cme/bodynems/data/output/test")
OUT_DIR =  pathlib.Path("/hits/fast/cme/bodynems/data/output/test")
dataset = "seqs_1"
threads = 1

config = {}
config["general"] = yaml.safe_load(open(pathlib.Path.cwd().parent / "configs/general.yaml"))
config["aligners"] = yaml.safe_load(open(pathlib.Path.cwd().parent / "configs/aligners.yaml"))
config["ensemble"] = yaml.safe_load(open(pathlib.Path.cwd().parent / "configs/ensemble.yaml"))

tool = "ClustalO"

in_file = OUT_DIR / "seqs_1" / "sequences.fasta"
out_dir = OUT_DIR / "seqs_1" / tool
log_file = OUT_DIR / "seqs_1" / tool / "ensemble.log"

manager_class:Manager = infer_manager(config["ensemble"][tool]["aligner"])
manager = manager_class(config, tool, in_file, out_dir, log_file, threads, shuffle=False)

os.makedirs(OUT_DIR / "seqs_1" / tool, exist_ok=True)
open(log_file, "a").close()

ensemble = manager.compute()
ensemble_dir = OUT_DIR / dataset / "ensemble"
os.makedirs(ensemble_dir, exist_ok=True)
manager.save_ensemble(ensemble, ensemble_dir)